<a href="https://colab.research.google.com/github/johntango/DeepSeek-R1-Medical/blob/main/DeepSpeedMedical001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from huggingface_hub import login
from google.colab import userdata

# Get HuggingFace token
hf_token = userdata.get('Lora')
login(hf_token)

In [2]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


In [6]:
import wandb
# get wandb.ai token
wb_token = userdata.get('WANDB_API_KEY')

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jrwtango (jrwtango-massachusetts-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [8]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

#Test and run model

In [9]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
{}"""



In [10]:
question = "A 55-year-old woman presents with a long history of urine leakage triggered by physical activities such as laughing or lifting heavy objects. She denies experiencing nocturnal enuresis or urgency. A gynecological exam and Q-tip test are performed. Based on these findings, what would cystometry most likely reveal regarding her post-void residual volume and detrusor muscle activity?"

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


Cystometry, also known as bladder diary, is a diagnostic tool used to assess how the bladder behaves during filling and emptying. In this case, the patient has a history of stress urinary incontinence (SUI) due to physical activities like laughing or lifting heavy objects. She denies having nocturnal enuresis or urgency, which are common in other types of urinary incontinence such as urge incontinence. 

Given that a gynecological exam and Q-tip test have already been performed, it's likely that these tests have identified the cause of her SUI. The Q-tip test checks for urethral obstruction, which can be a common cause of SUI in women. If the Q-tip test is positive, it indicates that the urethral lumen is partially or completely obstructed, leading to increased intravesicular pressure and thus contributing to the leakage.

Cystometry would show two main things: the post-void residual volume and detrusor muscle activity. Post-void residual volume refers to the amount of urine left in t